In [2]:
from data_processing import *
import tensorflow as tf
import copy
import numpy as np

In [3]:
dialogues, ides = [], []
for i in range(0, 9):
    dialogues.append(preprocessed_dialogue(i))
    ides.append(preprocessed_ide(i))

In [4]:
dialogue_input, dialogue_output = sort(dialogues)
ide_input, ide_output = sort(ides)
maxLength = 0
# 21 is the max length
for i in dialogue_input:
    maxLength = max(maxLength, len(i))

In [10]:
one_hot_encoded_dialogue_input, one_hot_encoded_dialogue_output = np.array([]), np.array([])
sample = []
# Trying padding and processing
for i in dialogue_input:
    temp = i
    while (len(temp) < 21): temp.append(0)
    sample.append(temp)
    
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
sample = scaler.fit_transform(sample)

one_hot_encoded_dialogue_input = tf.keras.utils.to_categorical(sample)

for i in dialogue_output:
    temp = tf.keras.utils.to_categorical(i).tolist()
    while (len(temp) < 47):
        temp.append(0)
    one_hot_encoded_dialogue_output = np.append(one_hot_encoded_dialogue_output, temp)
# print(one_hot_encoded_dialogue_input[0])

In [53]:
def build_model():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=21, activation='softmax'),
#         tf.keras.layers.LSTM(units=64, input_shape=(21, 43)),
        tf.keras.layers.LSTM(units=64, input_shape=(21, 43)),
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_and_evaluate_model(model, X, y, k=5):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=0)
        scores.append(score)
    return np.mean(scores)

# print()
# Load the input data and one-hot encode it
X = one_hot_encoded_dialogue_input
y = one_hot_encoded_dialogue_output

model = build_model()
score = train_and_evaluate_model(model, X, y)
print("Cross-validated accuracy:", score)


Cross-validated accuracy: 0.021634615771472455


In [58]:
# This is for dialogue input without one-hot encoding 
def build_model2():
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(units=21, activation='softmax'),
#         tf.keras.layers.LSTM(units=64, input_shape=(21, 43)),
        tf.keras.layers.LSTM(units=64, input_shape=(21, 43)),
        tf.keras.layers.Dense(units=21, activation='relu'),
        tf.keras.layers.Dense(units=1, activation='relu')
    ])
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

def train_and_evaluate_model2(model, X, y, k=5):
    from sklearn.model_selection import KFold
    kfold = KFold(n_splits=k, shuffle=True, random_state=0)
    scores = []
    for train_index, test_index in kfold.split(X):
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]
        model.fit(X_train, y_train, epochs=10, batch_size=32, verbose=0)
        score = model.evaluate(X_test, y_test, verbose=0)
        scores.append(score)
    return np.mean(scores)
model2 = build_model2()
print(len(sample), len(sample))
score_2 =  train_and_evaluate_model2(model, sample, np.array(dialogue_output))

2080 2080


ValueError: in user code:

    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1160, in train_function  *
        return step_function(self, iterator)
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1146, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 1135, in run_step  **
        outputs = model.train_step(data)
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\engine\training.py", line 993, in train_step
        y_pred = self(x, training=True)
    File "C:\Users\msok0\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
    File "C:\Users\msok0

In [ ]:
TypeError: only integer scalar arrays can be converted to a scalar index
    